# OWASP Top 10 (2021) - Secure Coding Guide

The **OWASP Top 10** is the most widely recognized standard for web application security risks. This notebook covers each vulnerability with practical Python examples showing both vulnerable and secure implementations.

---

## Vulnerability Prevalence Overview

Based on OWASP 2021 data, here's the prevalence of each vulnerability category across tested applications.

In [ ]:
import plotly.express as px
import pandas as pd

# OWASP Top 10 (2021) prevalence data
owasp_data = pd.DataFrame({
    'Vulnerability': [
        'A01: Broken Access Control',
        'A02: Cryptographic Failures',
        'A03: Injection',
        'A04: Insecure Design',
        'A05: Security Misconfiguration',
        'A06: Vulnerable Components',
        'A07: Auth Failures',
        'A08: Integrity Failures',
        'A09: Logging Failures',
        'A10: SSRF'
    ],
    'Incidence Rate (%)': [94.55, 46.44, 33.27, 24.19, 90.00, 27.96, 22.47, 16.67, 53.67, 2.72],
    'Rank': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
})

fig = px.bar(
    owasp_data,
    x='Vulnerability',
    y='Incidence Rate (%)',
    color='Incidence Rate (%)',
    color_continuous_scale='Reds',
    title='OWASP Top 10 (2021) - Vulnerability Prevalence',
    text='Incidence Rate (%)'
)
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(xaxis_tickangle=-45, showlegend=False, height=500)
fig

---

## A01: Broken Access Control

**Risk:** Users can act outside their intended permissions, accessing unauthorized data or functions.

**Impact:** Data theft, unauthorized modifications, privilege escalation.

In [ ]:
# ❌ VULNERABLE: No access control - any user can view any profile
from flask import Flask, request

app = Flask(__name__)

@app.route('/user/<user_id>')
def get_user_profile_vulnerable(user_id):
    # Directly returns user data without checking permissions
    user = db.query(f"SELECT * FROM users WHERE id = {user_id}")
    return user  # IDOR vulnerability - anyone can access any user's data

In [ ]:
# ✅ SECURE: Proper access control with authorization check
from flask import Flask, request, abort, session
from functools import wraps

def require_auth(f):
    @wraps(f)
    def decorated(*args, **kwargs):
        if 'user_id' not in session:
            abort(401)  # Unauthorized
        return f(*args, **kwargs)
    return decorated

@app.route('/user/<int:user_id>')
@require_auth
def get_user_profile_secure(user_id):
    current_user = session['user_id']
    
    # Check if user can access this resource
    if current_user != user_id and not is_admin(current_user):
        abort(403)  # Forbidden
    
    user = db.query("SELECT * FROM users WHERE id = ?", (user_id,))
    return user

---

## A02: Cryptographic Failures

**Risk:** Sensitive data exposed due to weak or missing encryption.

**Impact:** Credential theft, data breaches, regulatory violations.

In [ ]:
# ❌ VULNERABLE: Storing passwords in plaintext / weak hashing
import hashlib

def store_password_vulnerable(password):
    # MD5 is broken - vulnerable to rainbow table attacks
    hashed = hashlib.md5(password.encode()).hexdigest()
    return hashed

def encrypt_data_vulnerable(data):
    # Hardcoded key, weak algorithm
    key = "mysecretkey123"  # Never hardcode secrets!
    return simple_xor(data, key)

In [ ]:
# ✅ SECURE: Using bcrypt for passwords, proper encryption
import bcrypt
from cryptography.fernet import Fernet
import os

def store_password_secure(password: str) -> bytes:
    """Hash password with bcrypt (auto-salted, slow by design)"""
    salt = bcrypt.gensalt(rounds=12)
    return bcrypt.hashpw(password.encode(), salt)

def verify_password(password: str, hashed: bytes) -> bool:
    return bcrypt.checkpw(password.encode(), hashed)

def encrypt_data_secure(data: bytes) -> bytes:
    """Encrypt with Fernet (AES-128-CBC + HMAC)"""
    key = os.environ.get('ENCRYPTION_KEY')  # Load from env
    cipher = Fernet(key)
    return cipher.encrypt(data)

---

## A03: Injection

**Risk:** Untrusted data sent to an interpreter as part of a command or query.

**Impact:** Data loss, data corruption, complete system takeover.

In [ ]:
# ❌ VULNERABLE: SQL Injection via string concatenation
import sqlite3

def get_user_vulnerable(username):
    conn = sqlite3.connect('users.db')
    cursor = conn.cursor()
    # User input directly in query - SQL injection!
    query = f"SELECT * FROM users WHERE username = '{username}'"
    # Attack: username = "' OR '1'='1" returns all users
    # Attack: username = "'; DROP TABLE users; --" deletes table
    cursor.execute(query)
    return cursor.fetchone()

In [ ]:
# ✅ SECURE: Parameterized queries prevent injection
import sqlite3

def get_user_secure(username):
    conn = sqlite3.connect('users.db')
    cursor = conn.cursor()
    # Parameterized query - input treated as data, not code
    query = "SELECT * FROM users WHERE username = ?"
    cursor.execute(query, (username,))
    return cursor.fetchone()

# For ORMs like SQLAlchemy - also safe
from sqlalchemy.orm import Session
def get_user_orm(db: Session, username: str):
    return db.query(User).filter(User.username == username).first()

---

## A04: Insecure Design

**Risk:** Missing or ineffective security controls in the application design.

**Impact:** Business logic flaws, abuse of features, data exposure.

In [ ]:
# ❌ VULNERABLE: No rate limiting on sensitive operations
@app.route('/forgot-password', methods=['POST'])
def forgot_password_vulnerable():
    email = request.form['email']
    # No rate limiting - allows credential stuffing/enumeration
    user = User.query.filter_by(email=email).first()
    if user:
        send_reset_email(user)  # Attacker can enumerate valid emails
        return "Reset email sent"
    return "User not found"  # Information disclosure!

In [ ]:
# ✅ SECURE: Rate limiting + consistent responses
from flask_limiter import Limiter
from flask_limiter.util import get_remote_address

limiter = Limiter(app, key_func=get_remote_address)

@app.route('/forgot-password', methods=['POST'])
@limiter.limit("3 per minute")  # Rate limit
def forgot_password_secure():
    email = request.form['email']
    user = User.query.filter_by(email=email).first()
    
    if user:
        send_reset_email(user)
    
    # Same response regardless of user existence - prevents enumeration
    return "If an account exists, a reset email will be sent."

---

## A05: Security Misconfiguration

**Risk:** Insecure default configurations, incomplete setups, open cloud storage.

**Impact:** Unauthorized access, data exposure, system compromise.

In [ ]:
# ❌ VULNERABLE: Debug mode, default credentials, verbose errors
from flask import Flask

app = Flask(__name__)
app.config['DEBUG'] = True  # Exposes stack traces in production!
app.config['SECRET_KEY'] = 'dev'  # Weak/default secret

# Verbose error messages expose internals
@app.errorhandler(500)
def handle_error_vulnerable(error):
    return f"Error: {str(error)}\nStack: {traceback.format_exc()}"

In [ ]:
# ✅ SECURE: Production-ready configuration
import os
from flask import Flask

app = Flask(__name__)

# Load from environment, fail if missing
app.config['DEBUG'] = False
app.config['SECRET_KEY'] = os.environ['SECRET_KEY']  # Strong, random key
app.config['SESSION_COOKIE_SECURE'] = True
app.config['SESSION_COOKIE_HTTPONLY'] = True
app.config['SESSION_COOKIE_SAMESITE'] = 'Lax'

@app.errorhandler(500)
def handle_error_secure(error):
    # Log full error internally, return generic message to user
    app.logger.error(f"Internal error: {error}")
    return "An error occurred. Please try again later.", 500

---

## A06: Vulnerable and Outdated Components

**Risk:** Using libraries/frameworks with known vulnerabilities.

**Impact:** Full range of weaknesses including injection, broken access control.

In [ ]:
# ❌ VULNERABLE: Outdated dependencies with known CVEs
# requirements.txt with vulnerable versions:
# Django==2.2.0        # CVE-2019-14232, CVE-2019-14233
# PyYAML==5.1          # CVE-2020-1747 (arbitrary code execution)
# Pillow==6.0.0        # Multiple CVEs

import yaml
# Vulnerable: yaml.load allows arbitrary Python execution
data = yaml.load(user_input)  # RCE vulnerability!

In [ ]:
# ✅ SECURE: Regular updates + safe loading
# Use pip-audit, safety, or dependabot for vulnerability scanning
# pip install pip-audit && pip-audit

import yaml

# Safe loader prevents arbitrary code execution
data = yaml.safe_load(user_input)

# Pin dependencies with hashes for integrity
# requirements.txt:
# Django==4.2.7 \
#     --hash=sha256:abc123...

# Automated dependency updates with Dependabot/Renovate

---

## A07: Identification and Authentication Failures

**Risk:** Weak authentication mechanisms, session management flaws.

**Impact:** Account takeover, identity theft, unauthorized access.